# Imports

In [65]:
import json
import requests

from datetime import datetime, timedelta, timezone

import pandas as pd
import numpy as np
from geopy.distance import distance

import math
import pprint
import time

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from typing import List, Union

# Query GraphQL

In [66]:
def query_graphql(start_time: int, end_time: int, route: str) -> list:
    query = f"""{{
        trynState(agency: "muni",
                  startTime: "{start_time}",
                  endTime: "{end_time}",
                  routes: ["{route}"]) {{
            agency
            startTime
            routes {{
                stops {{
                    sid
                    lat
                    lon
                }}
                routeStates {{
                    vtime
                    vehicles {{
                        vid
                        lat
                        lon
                        did
                    }}
                }}
            }}
        }}
    }}
    """
    query_url = f"https://06o8rkohub.execute-api.us-west-2.amazonaws.com/dev/graphql?query={query}"

    request = requests.get(query_url).json()
    try:
        return request['data']['trynState']['routes']
    except KeyError:
        return None

# Produce Datatables

In [67]:
def produce_stops(data: list, route: str) -> pd.DataFrame:
    stops = pd.io.json.json_normalize(data,
                                      record_path=['stops']) \
            .rename(columns={'lat': 'LAT',
                             'lon': 'LON',
                             'sid': 'SID'}) \
            .reindex(['SID', 'LAT', 'LON'], axis='columns')
    
    # obtain stop directions
    stops['DID'] = stops['SID'].map({stop: direction['id']
                                     for direction in requests
                                                      .get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}")
                                                      .json()['directions']
                                     for stop in direction['stops']})
    
    # remove stops that don't have an associated direction
    stops = stops.dropna(axis='index', subset=['DID'])
    
    # obtain stop ordinals
    stops['ORD'] = stops['SID'].map({stop_meta['id']: ordinal
                                     for ordinal, stop_meta
                                     in enumerate(requests
                                                  .get("http://restbus.info/api/agencies/sf-muni/"
                                                       f"routes/{route}")
                                                  .json()['stops'])})
    
    return stops

In [68]:
def produce_buses(data: list) -> pd.DataFrame:
     return pd.io.json.json_normalize(data,
                                      record_path=['routeStates', 'vehicles'],
                                      meta=[['routeStates', 'vtime']]) \
            .rename(columns={'lat': 'LAT',
                             'lon': 'LON',
                             'vid': 'VID',
                             'did': 'DID',
                             'routeStates.vtime': 'TIME'}) \
            .reindex(['TIME', 'VID', 'LAT', 'LON', 'DID'], axis='columns')

# Eclipses

In [69]:

# haversine formula for calcuating distance between two coordinates in lat lon
# from bird eye view; seems to be +- 8 meters difference from geopy distance
def haver_distance(latstop,lonstop,latbus,lonbus):

    latstop,lonstop,latbus,lonbus = map(np.deg2rad,[latstop,lonstop,latbus,lonbus])
    eradius = 6371000
    
    latdiff = (latbus-latstop)
    londiff = (lonbus-lonstop)
    
    a = np.sin(latdiff/2)**2 + np.cos(latstop)*np.cos(latbus)*np.sin(londiff/2)**2
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    
    distance = eradius*c
    return distance

def find_eclipses(buses, stop):
    """
    Find movement of buses relative to the stop, in distance as a function of time.
    """
    def split_eclipses(eclipses, threshold=30*60*1000) -> List[pd.DataFrame]:
        """
        Split buses' movements when they return to a stop after completing the route.
        """
        disjoint_eclipses = []
        for bus_id in eclipses['VID'].unique(): # list of unique VID's
            # obtain distance data for this one bus
            bus = eclipses[eclipses['VID'] == bus_id].sort_values('TIME')
            #pprint.pprint(bus)
            #pprint.pprint(bus['TIME'].shift())
            #pprint.pprint(bus['TIME'].shift() + threshold)
            #print('===============')
            # split data into groups when there is at least a `threshold`-ms gap between data points
            group_ids = (bus['TIME'] > (bus['TIME'].shift() + threshold)).cumsum()

            # store groups
            for _, group in bus.groupby(group_ids):
                disjoint_eclipses.append(group)
        return disjoint_eclipses

    eclipses = buses.copy()
    #eclipses['DIST'] = eclipses.apply(lambda row: distance(stop[['LAT','LON']],row[['LAT','LON']]).meters,axis=1)
    
    stopcord = stop[['LAT', 'LON']]
    buscord = eclipses[['LAT', 'LON']]

    # calculate distances fast with haversine function 
    eclipses['DIST'] = haver_distance(stopcord['LAT'],stopcord['LON'],buscord['LAT'],buscord['LON'])
    # only keep positions within 750 meters within the given stop; (filtering out)
    eclipses = eclipses[eclipses['DIST'] < 750]
    
    # update the coordinates list 
    stopcord = stop[['LAT', 'LON']].values
    buscord = eclipses[['LAT', 'LON']].values
    
    # calculate distances again using geopy for the distance<750m values, because geopy is probably more accurate
    dfromstop = []
    for row in buscord:
        busdistance = distance(stopcord,row).meters
        dfromstop.append(busdistance)
    eclipses['DIST'] = dfromstop
    
    # for haversine function:
    #stopcord = stop[['LAT', 'LON']]
    #buscord = eclipses[['LAT', 'LON']]
    #eclipses['DIST'] = haver_distance(stopcord['LAT'],stopcord['LON'],buscord['LAT'],buscord['LON'])
    
    eclipses['TIME'] = eclipses['TIME'].astype(np.int64)
    eclipses = eclipses[['TIME', 'VID', 'DIST']]
    
    eclipses = split_eclipses(eclipses)
    
    return eclipses

def find_nadirs(eclipses):
    """
    Find points where buses are considered to have encountered the stop.
    
    Nadir is an astronomical term that describes the lowest point reached by an orbiting body.
    """
    def calc_nadir(eclipse: pd.DataFrame) -> Union[pd.Series, None]:
        nadir = eclipse.iloc[eclipse['DIST'].values.argmin()]
        if nadir['DIST'] < 100:  # if min dist < 100, then reasonable candidate for nadir
            return nadir
        else:  # otherwise, hardcore datasci is needed
            rev_eclipse = eclipse.iloc[::-1]
            rev_nadir = rev_eclipse.iloc[rev_eclipse['DIST'].values.argmin()]
            if nadir['TIME'] == rev_nadir['TIME']:  # if eclipse has a global min
                return nadir  # then it's the best candidate for nadir
            else:  # if eclipse's min occurs at two times
                mid_nadir = nadir.copy()
                mid_nadir['DIST'] = (nadir['DIST'] + rev_nadir['DIST'])/2
                return mid_nadir  # take the midpoint of earliest and latest mins
    
    nadirs = []
    for eclipse in eclipses:
        nadirs.append(calc_nadir(eclipse)[['VID', 'TIME']])
        
    return pd.DataFrame(nadirs)
            
    
def show_stop(eclipses, nadirs):
    fig = plt.figure(figsize=(18, 9))
    ax = fig.add_subplot(111)

    for eclipse in eclipses:
        plt.plot(*eclipse[['TIME', 'DIST']].values.T)
        
    for nadir_time in nadirs['TIME']:
        plt.axvline(nadir_time, linestyle='--', linewidth=.5)

    # format plot
    ax.get_xaxis().set_major_formatter(  # convert x-axis tick labels to time of day
        FuncFormatter(lambda x, p: datetime.fromtimestamp(int(x)//1000).strftime('%I:%M%p')))
    plt.title(f"Eclipses at Stop {stop_id}"
              f" from {datetime.fromtimestamp(int(start_time)//1000).strftime('%a %b %d %I:%M%p')}"
              f" to {datetime.fromtimestamp(int(end_time)//1000).strftime('%a %b %d %I:%M%p')}")
    plt.xlabel("Time")
    plt.ylabel("Distance from Stop (meters)")

    plt.show()

# Hardcore Data Science

In [70]:
route = ["12", "14"]

timespan = ("08:00",
            "11:00")

dates = [
    "2018-11-12",
    "2018-11-13",
    "2018-11-14",
    "2018-11-15",
    "2018-11-16",
]

Issue: The plot is labeled based on the machine's current timezone, which may not necessarily match the times sent to the API. To remedy this, the logic for displaying the plot would have to be adjusted to account for the UTC offset of the epochs we get back from the API, which I'm hoping there's a module for but I'm not presently familiar with any.

Issue: currently, if a trajectory looks like `/~V`, the left edge is selected as the nadir. Based on the data, I suspect that the initial upslope may be a GPS glitch as it's being initialized, I believe the trough on the right should be selected instead.

In [71]:
class BusData:
    def __init__(self):
        self.data = {}
    
    @property
    def routes(self):
        return list(self.data.keys())
    
    @property
    def stops(self, route_id):
        return list(self.data.get(route_id, {}).keys())
    
    def append(self, other_data):
        for route_id, other_route in other_data.items():
            route = self.data.get(route_id)
            if route:
                for stop_id, other_stop in other_route.items():
                    stop = route.get(stop_id)
                    if stop:
                        stop['eclipses'].extend(other_stop['eclipses'])
                    else:
                        route[stop_id] = other_stop
            else:
                self.data[route_id] = other_route
    
    @classmethod
    def read_file(cls, filename):
        bus_data = cls()
        with open(filename, 'r') as f:
            bus_data.append(json.load(f))
        return bus_data
                
    
    def write_file(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.data, f)

`BusData.data` specification:
```
{
    route_id: {  # route_id is a str
        stop_id: {  # stop_id is a str
            direction_id: str,
            order: int,
            lat: float,
            lon: float,
            eclipses: [
                {
                    bus_id: int,
                    timestamp: int,
                },
                {
                    bus_id: int,
                    timestamp: int,
                },
                ...
            ]
        },
        ...
    },
    ...
}
```

In [72]:
# grab a couple of sequential stops to look at
# get_stop_times(date, stop [time, route, dir])
# returns a df w/columns: vID, date, time, stop, route, dir
# ISSUE: really slow, any way to speed up graphQL query?

# routes = pd.DataFrame(columns = ["VID", "TIME", "SID", "DID", "ROUTE"])

# stop_ids = [stop['id']
#             for stop
#             in requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}").json()['stops']][2:4]

# bus_data = BusData()

# for stop_id in stop_ids:
#     for date in dates:
#         start_time = int(datetime.strptime(f"{date} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000
#         end_time   = int(datetime.strptime(f"{date} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000

#         data = query_graphql(start_time, end_time, route)

#         if data is None:  # API might refuse to cooperate
#             print("API probably timed out")
#             continue
#         elif len(data) == 0:  # some days somehow have no data
#             print(f"no data for {month}/{day}")
#             continue
#         else:
#             stops = produce_stops(data)
#             buses = produce_buses(data)

#             stop = stops[stops['SID'] == stop_id].squeeze()
#             buses = buses[buses['DID'] == stop['DID']]

#             eclipses = find_eclipses(buses, stop)
#             nadirs = find_nadirs(eclipses)
#             nadirs["TIME"] = nadirs["TIME"].apply(lambda x: datetime.fromtimestamp(x//1000, timezone(timedelta(hours = -8))).strftime('%a %b %d %I:%M%p'))
#             nadirs["SID"] = stop_id
#             nadirs["DID"] = stop["DID"]
#             nadirs["ROUTE"] = route
#             routes = routes.append(nadirs)


#             show_stop(eclipses, nadirs)

#             bus_data.append({
#                 route: {
#                     stop_id: {
#                         'direction_id': stop['DID'],
#                         'order': int(stop['ORD']),
#                         'lat': stop['LAT'],
#                         'lon': stop['LON'],
#                         'eclipses': [
#                             {
#                                 'bus_id': bus_id,
#                                 'timestamp': int(timestamp)
#                             }
#                             for bus_id, timestamp in zip(nadirs['VID'].tolist(),
#                                                          nadirs['TIME'].tolist())
#                         ]
#                     }
#                 }
#             })
        
# bus_data.write_file("bus_data.json")
                      

In [73]:
# get_stops
# ------------------------------------------------------------------------------------------
# parameters:
# dates: an array of dates, formatted as strings in the form YYYY-MM-DD
# routes: an array of routes, each represented as a string
# directions: an array of strings representing the directions to filter
# stops: an array of strings representing the stops to filter
# times: a tuple with the start and end times (in UTC -8:00) as strings in the form HH:MM 
# 
# returns:
# stops: a DataFrame, filtered by the given directions and stops, with the following columns:
# VID: the vehicle ID
# Time: a datetime object representing the date/time of the stop
# Route: the route on which the stop occurred
# Stop: the stop at which the stop occurred
# Dir: the direction in which the stop occurred
# -------------------------------------------------------------------------------------------
def get_stops(dates, routes, directions = [], new_stops = [], times = ("00:00", "23:59")):
    bus_stops = pd.DataFrame(columns = ["VID", "TIME", "SID", "DID", "ROUTE"])
    
    for route in routes:
        stop_ids = [stop['id']
            for stop
            in requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}").json()['stops']][2:4]

        for stop_id in stop_ids:
            # check if stops to filter were provided, or if the stop_id is in the list of filtered stops
            if (stop_id in new_stops) ^ (len(new_stops) == 0):
                for date in dates:
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: starting processing on stop {stop_id} on route {route} on {date}.")
                    start_time = int(datetime.strptime(f"{date} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000
                    end_time   = int(datetime.strptime(f"{date} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000

                    data = query_graphql(start_time, end_time, route)
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: performed query.")
                          
                    if data is None:  # API might refuse to cooperate
                        print("API probably timed out")
                        continue
                    elif len(data) == 0:  # some days somehow have no data
                        print(f"no data for {month}/{day}")
                        continue
                    else:
                        stops = produce_stops(data, route)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced stops.")
                        #pprint.pprint(stops)
                              
                        buses = produce_buses(data)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced buses.")
                        #pprint.pprint(buses)
                        
                        # select single stop that match stop_id
                        stop = stops[stops['SID'] == stop_id].squeeze()
                        # select buses that have matching DID with the stop. note* this will not select inbound
                        buses = buses[buses['DID'] == stop['DID']]
                        
                        #pprint.pprint(buses)
                        #pprint.pprint(stop)
                        
                        starttime = time.time()
                        eclipses = find_eclipses(buses, stop)
                        print('time elapsed for eclipse function: %s' % (time.time() - starttime))
                              
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found eclipses.")
                              
                        nadirs = find_nadirs(eclipses)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found nadirs.")
                            
                        nadirs["TIME"] = nadirs["TIME"].apply(lambda x: datetime.fromtimestamp(x//1000, timezone(timedelta(hours = -8))).strftime('%a %b %d %Y %I:%M%p'))
                        nadirs["SID"] = stop_id
                        nadirs["DID"] = stop["DID"]
                        nadirs["ROUTE"] = route
                        old_length = len(bus_stops)
                        bus_stops = bus_stops.append(nadirs, sort = True)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finished processing.")

    # filter for directions
    if len(directions) > 0:
        bus_stops = bus_stops.loc[bus_stops['DID'].apply(lambda x: x in directions)]
    
    return bus_stops

In [74]:
new_stops = get_stops(dates, route, directions = ['14___O_F00'], new_stops = ['5528'], times = timespan)

Mon Jan 28 02:14:41 PM: starting processing on stop 5528 on route 14 on 2018-11-12.
Mon Jan 28 02:14:52 PM: performed query.
Mon Jan 28 02:14:53 PM: produced stops.
Mon Jan 28 02:14:53 PM: produced buses.
before haversine 1548713693.291006
after haversine 1548713693.296182
before geopy 1548713693.296296
time elapsed for eclipse function: 2.8825478553771973
Mon Jan 28 02:14:56 PM: found eclipses.
Mon Jan 28 02:14:56 PM: found nadirs.
Mon Jan 28 02:14:56 PM: finished processing.
Mon Jan 28 02:14:56 PM: starting processing on stop 5528 on route 14 on 2018-11-13.
Mon Jan 28 02:15:04 PM: performed query.
Mon Jan 28 02:15:05 PM: produced stops.
Mon Jan 28 02:15:05 PM: produced buses.
before haversine 1548713705.874632
after haversine 1548713705.879687
before geopy 1548713705.879767
time elapsed for eclipse function: 2.660742998123169
Mon Jan 28 02:15:08 PM: found eclipses.
Mon Jan 28 02:15:08 PM: found nadirs.
Mon Jan 28 02:15:08 PM: finished processing.
Mon Jan 28 02:15:08 PM: starting proc

In [75]:
# TODO: parse direction as inbound/outbound? (remove route indicator)
# filter by direction/stop if provided
# split date/time
new_stops

,DID,ROUTE,SID,TIME,VID
254,14___O_F00,14,5528,Mon Nov 12 2018 08:03AM,7225
10499,14___O_F00,14,5528,Mon Nov 12 2018 10:29AM,7225
974,14___O_F00,14,5528,Mon Nov 12 2018 08:14AM,7254
10843,14___O_F00,14,5528,Mon Nov 12 2018 10:34AM,7254
1460,14___O_F00,14,5528,Mon Nov 12 2018 08:21AM,7244
11577,14___O_F00,14,5528,Mon Nov 12 2018 10:44AM,7244
2107,14___O_F00,14,5528,Mon Nov 12 2018 08:30AM,7272
12159,14___O_F00,14,5528,Mon Nov 12 2018 10:53AM,7272
3342,14___O_F00,14,5528,Mon Nov 12 2018 08:49AM,7232
3658,14___O_F00,14,5528,Mon Nov 12 2018 08:54AM,7229


In [76]:
new_stops['DID'].unique()

array(['14___O_F00'], dtype=object)

In [77]:
new_stops['SID'].unique()

array(['5528'], dtype=object)

In [78]:
def average_waiting_time(df, start_time, end_time):
    minute_range = [start_time.replace(minute = start_time.minute + i) for i in range(end_time.minute - start_time.minute)]
    wait_times = pd.DataFrame(columns = ["ROUTE", "TIME", "WAIT"])
    
    for minute in minute_range:
        print("hi")

In [79]:
new_stops["timestamp"] = new_stops["TIME"].apply(lambda x: datetime.strptime(x, '%a %b %d %Y %I:%M%p').timestamp())

In [80]:
new_stops['date'] = new_stops["TIME"].apply(lambda x: datetime.strptime(x, '%a %b %d %Y %I:%M%p').date())

In [81]:
pivot = new_stops[['date', 'timestamp']].pivot_table(values = ['timestamp'], index = ['date'])

In [82]:
pivot['timestamp'] = pivot['timestamp'].apply(lambda x: )

SyntaxError: invalid syntax (<ipython-input-82-f1e297e488d5>, line 1)